In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
llm_config = {"model": "gpt-3.5-turbo"}

Define a code executor

In [2]:
#free style coding + code execution
#this is code executor - there are lot of executors including docker and jupyter #based 
from autogen.coding import LocalCommandLineCodeExecutor

In [3]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
)

Create agents

In [4]:
from autogen import ConversableAgent, AssistantAgent

1. Agent with code executor configuration

In [5]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

2. Agent with code writing capability

In [6]:
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
code_writer_agent_system_message = code_writer_agent.system_message

In [8]:
print(code_writer_agent_system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

The task!

Ask the two agents to collaborate on a stock analysis task.

In [13]:
import datetime

today = datetime.datetime.now().date()
message = f"Today is {today}. "\
"Create a plot showing stock gain YTD for NVDA and TLSA. "\
"Make sure the code is in markdown code block and save the figure"\
" to a file ytd_stock_gains.png."""

In [12]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message,
)

code_executor_agent (to code_writer_agent):

Today is 2024-07-05. Create a plot showing stock gain YTD for NVDA and TLSA. Make sure the code is in markdown code block and save the figure to a file ytd_stock_gains.png.

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

```python
# filename: ytd_stock_gains.py
import matplotlib.pyplot as plt

# Stock gains YTD for NVDA and TLSA (hypothetical values)
stock_prices_nvda = [210, 240, 260, 300, 280, 310, 330]
stock_prices_tlsa = [400, 420, 380, 410, 430, 450, 460]

# Calculate YTD stock gains
stock_gains_nvda = [(price - stock_prices_nvda[0]) / stock_prices_nvda[0] * 100 for price in stock_prices_nvda]
stock_gains_tlsa = [(price - stock_prices_tlsa[0]) / stock_prices_tlsa[0] * 100 for price in stock_prices_tlsa]

# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(stock_gains_nvda, label='NVDA')
plt.plot(stock_gains_tlsa, label='TLSA')
plt.title('Stock Gain YTD Compa

KeyboardInterrupt: Interrupted by user

In [14]:
import os
from IPython.display import Image

Image(os.path.join("coding", "ytd_stock_gains.png"))

FileNotFoundError: No such file or directory: 'coding/ytd_stock_gains.png'

FileNotFoundError: No such file or directory: 'coding/ytd_stock_gains.png'

<IPython.core.display.Image object>

User-Defined Functions

Instead of asking LLM to generate the code for downloading stock data and plotting charts each time, you can define functions for these two tasks and have LLM call these functions in the code.

In [15]:
def get_stock_prices(stock_symbols, start_date, end_date):
    """Get the stock prices for the given stock symbols between
    the start and end dates.

    Args:
        stock_symbols (str or list): The stock symbols to get the
        prices for.
        start_date (str): The start date in the format 
        'YYYY-MM-DD'.
        end_date (str): The end date in the format 'YYYY-MM-DD'.
    
    Returns:
        pandas.DataFrame: The stock prices for the given stock
        symbols indexed by date, with one column per stock 
        symbol.
    """
    import yfinance

    stock_data = yfinance.download(
        stock_symbols, start=start_date, end=end_date
    )
    return stock_data.get("Close")

In [16]:
def plot_stock_prices(stock_prices, filename):
    """Plot the stock prices for the given stock symbols.

    Args:
        stock_prices (pandas.DataFrame): The stock prices for the 
        given stock symbols.
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    for column in stock_prices.columns:
        plt.plot(
            stock_prices.index, stock_prices[column], label=column
                )
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.savefig(filename)

Create a new executor with the user-defined functions

In [17]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)

In [18]:
code_writer_agent_system_message += executor.format_functions_for_prompt()
print(code_writer_agent_system_message)

You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feed

Let's update the agents with the new system message

In [19]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [20]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

Start the same task again!

In [21]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}."
    "Download the stock prices YTD for NVDA and TSLA and create"
    "a plot. Make sure the code is in markdown code block and "
    "save the figure to a file stock_prices_YTD_plot.png.",
)

code_executor_agent (to code_writer_agent):

Today is 2024-07-05.Download the stock prices YTD for NVDA and TSLA and createa plot. Make sure the code is in markdown code block and save the figure to a file stock_prices_YTD_plot.png.

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

```python
from functions import get_stock_prices, plot_stock_prices
import pandas as pd
import datetime

# Define the start date as the beginning of the year
start_date = datetime.datetime(datetime.datetime.now().year, 1, 1).strftime('%Y-%m-%d')
end_date = '2024-07-05'

stock_symbols = ['NVDA', 'TSLA']

stock_prices = get_stock_prices(stock_symbols, start_date, end_date)

plot_stock_prices(stock_prices, 'stock_prices_YTD_plot.png')
```
After executing this code, the stock prices year-to-date for NVDA and TSLA will be downloaded and a plot will be created. The plot will be saved as `stock_prices_YTD_plot.png`.

If there are any issue

In [22]:
#PLOT the results
Image(os.path.join("coding", "stock_prices_YTD_plot.png"))

FileNotFoundError: No such file or directory: 'coding/stock_prices_YTD_plot.png'

FileNotFoundError: No such file or directory: 'coding/stock_prices_YTD_plot.png'

<IPython.core.display.Image object>